# Выбор «самой уверенной» AlphaFold-модели в каждой подпапке

В этом ноутбуке мы будем для каждой папки типа `../results/alphafold_models/fold_XXXX/`:
1. Искать все файлы `*_summary_confidences_{i}.json` (i = 0…4).
2. Из каждого JSON пытается извлечь (в порядке приоритета):
   - `ranking_score` (самая важная метрика в вашем случае);
   - если нет `ranking_score`, то `ptm`;
   - если нет и того, и другого — выводит предупреждение.
3. Определять индекс `i_best` (0–4), у которого `mean_plddt` максимален.
4. Выводить результирующую таблицу: `<имя_папки> → i_best, mean_plddt_best`.

В этом ноутбуке:
- Мы используем стандартный модуль `os` для обхода папок.
- Модуль `json` для чтения JSON.
- (При желании) `pandas` для аккуратного вывода таблицы.

In [1]:
import os
import json
from pathlib import Path
import pandas as pd


In [2]:
ROOT_DIR = Path("../results/alphafold_models")

## 1. Функция для чтения среднего pLDDT из summary_confidences_X.json

Внутри каждого `summary_confidences_{i}.json` AlphaFold обычно кладёт ключ `"plddt"` (список из pLDDT для каждого остатка) и/или уже рассчитывает `"ranking_score"`.  

Пытаемся найти ключи в порядке:
1. `ranking_score`  
2. `ptm`  
Если ни того, ни другого нет — возвращаем `None`.  

In [3]:
def read_confidence_from_summary(json_path: Path) -> float:
    """
    Читает из summary_confidences_{i}.json:
      1) Если есть "ranking_score" → возвращает его.
      2) Если нет, но есть "ptm" → возвращает его.
      Иначе возвращает None.
      
    Также выводит все ключи, если не найдено ни одного нужного.
    """
    try:
        with open(json_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"  [!] Ошибка при чтении {json_path.name}: {e}")
        return None

    # Если есть ranking_score
    if "ranking_score" in data:
        return float(data["ranking_score"])
    # Иначе если есть ptm
    if "ptm" in data:
        return float(data["ptm"])

    # Если ни то, ни другое — выведем ключи для отладки
    print(f"  [!] В {json_path.name} нет ни 'ranking_score', ни 'ptm'. Ключи файла:", list(data.keys()))
    return None


## 2. Проход по всем подпапкам и выбор лучшей модели

Алгоритм:
1. Перебрать все элементы `ROOT_DIR`, отфильтровать только директории.
2. Внутри каждой директории собрать список файлов `*_summary_confidences_{i}.json`.
3. Для каждого i (0…4) прочитать mean_plddt.
4. Найти `i_best = argmax(mean_plddt)`.
5. Сохранить в список (имя_папки, i_best, mean_plddt_best).


In [5]:
results = []  # сюда накопим (имя_папки, best_index, best_score)

# Проверяем наличие каталога
if not ROOT_DIR.exists():
    raise FileNotFoundError(f"Каталог {ROOT_DIR} не найден. Проверьте путь.")

for subdir in sorted(ROOT_DIR.iterdir()):
    if not subdir.is_dir():
        continue

    print(f"\n=== Обрабатываем папку: {subdir.name} ===")
    # Находим все summary_confidences_*.json
    summary_files = sorted(subdir.glob("*_summary_confidences_*.json"))
    print(f"Найдено {len(summary_files)} файлов summary_confidences:")
    for sf in summary_files:
        print("   ", sf.name)

    if len(summary_files) == 0:
        print(f"  [!] В папке {subdir.name} нет summary_confidences_*.json → пропускаем.")
        continue

    best_idx = None
    best_score = None

    for sf in summary_files:
        # Извлекаем индекс i из имени, например: fold_abil_cp003678_summary_confidences_2.json
        stem = sf.stem  # "fold_abil_cp003678_summary_confidences_2"
        try:
            idx = int(stem.split("_")[-1])
        except ValueError:
            print(f"  [!] Не удалось распарсить индекс из имени {sf.name}, пропускаем.")
            continue

        score = read_confidence_from_summary(sf)
        if score is None:
            continue

        print(f"    Модель {idx}: confidence = {score:.4f}")

        # Обновляем «лучший»
        if (best_score is None) or (score > best_score):
            best_score = score
            best_idx = idx

    if best_idx is None:
        print(f"  [!] Ни одна модель не дала валидного confidence → пропускаем папку {subdir.name}.")
        continue

    print(f"  => Лучшая модель: индекс {best_idx}, confidence = {best_score:.4f}")
    results.append((subdir.name, best_idx, best_score))



=== Обрабатываем папку: fold_abil_cp003678 ===
Найдено 5 файлов summary_confidences:
    fold_abil_cp003678_summary_confidences_0.json
    fold_abil_cp003678_summary_confidences_1.json
    fold_abil_cp003678_summary_confidences_2.json
    fold_abil_cp003678_summary_confidences_3.json
    fold_abil_cp003678_summary_confidences_4.json
    Модель 0: confidence = 0.9300
    Модель 1: confidence = 0.9200
    Модель 2: confidence = 0.9200
    Модель 3: confidence = 0.9200
    Модель 4: confidence = 0.9200
  => Лучшая модель: индекс 0, confidence = 0.9300

=== Обрабатываем папку: fold_abil_nz_cp120635 ===
Найдено 5 файлов summary_confidences:
    fold_abil_nz_cp120635_summary_confidences_0.json
    fold_abil_nz_cp120635_summary_confidences_1.json
    fold_abil_nz_cp120635_summary_confidences_2.json
    fold_abil_nz_cp120635_summary_confidences_3.json
    fold_abil_nz_cp120635_summary_confidences_4.json
    Модель 0: confidence = 0.9300
    Модель 1: confidence = 0.9300
    Модель 2: confiden

In [6]:
# Итоговый вывод
print("\n=== Результаты ===")
df = pd.DataFrame(results, columns=["folder", "best_model_index", "best_confidence"])
display(df)


=== Результаты ===


,folder,best_model_index,best_confidence
0,fold_abil_cp003678,0,0.93
1,fold_abil_nz_cp120635,0,0.93
2,fold_abil_oz007026,0,0.87
3,fold_ea59ptua_nz_cp069280,0,0.81
4,fold_ea59ptua_nz_cp147916,0,0.87
5,fold_ea59ptua_oy282455,0,0.81
6,fold_ea59ptuagaja_nz_lt634361,0,0.85
7,fold_ea59ptuapdc_m01a_cp034683,0,0.78
8,fold_ea59ptuapdc_m01a_cp045340,0,0.79
9,fold_ea59ptuapdc_m01a_nz_cp151867,0,0.77


## Анализ качества AlphaFold-моделей

Сводный результат по каждой папке: индекс лучшей модели всегда 0.

### Распределение confidence

**Всего белков**: 39  
- **Высокая уверенность (≥ 0.90)**: 7 моделей  
- **Умеренная уверенность (0.70 – 0.90)**: 29 моделей   
- **Низкая уверенность (< 0.70)**: 3 модели

1. **Очень надёжные модели (> 90 %)**  
   Эти структуры демонстрируют наивысший уровень доверия и будут взяты для дальнейшей работы
2. **Умеренно надёжные модели (70 % – 90 %)**  
Большинство моделей (29/39) относятся к этой категории. Эти белки достаточно качественно предсказаны и будут взяты для дальнейшей работы.
3. **Модели с низким доверием (< 70 %)**  
   Всего 3 модели (≈ 8 %) попали в эту категорию. Они имеют низкий уровень предсказанного качества и будут удалены для дальнейшей работы.  
   - `fold_hecs_ow818675` (0.67)  
   - `fold_jetc_wp_144121057` (0.50)  
   - `fold_nsnb_mcy4307946` (0.64)  